## Giới thiệu

Bài học này sẽ bao gồm:  
- Gọi hàm là gì và các trường hợp sử dụng của nó  
- Cách tạo một cuộc gọi hàm sử dụng OpenAI  
- Cách tích hợp cuộc gọi hàm vào một ứng dụng  

## Mục tiêu học tập

Sau khi hoàn thành bài học này, bạn sẽ biết cách và hiểu được:  

- Mục đích của việc sử dụng gọi hàm  
- Cài đặt Cuộc gọi Hàm sử dụng Dịch vụ OpenAI  
- Thiết kế các cuộc gọi hàm hiệu quả cho trường hợp sử dụng ứng dụng của bạn  


## Hiểu về Gọi Hàm

Trong bài học này, chúng ta muốn xây dựng một tính năng cho startup giáo dục của mình cho phép người dùng sử dụng chatbot để tìm các khóa học kỹ thuật. Chúng tôi sẽ đề xuất các khóa học phù hợp với trình độ kỹ năng, vai trò hiện tại và công nghệ mà họ quan tâm.

Để hoàn thành điều này, chúng ta sẽ sử dụng kết hợp:
 - `OpenAI` để tạo trải nghiệm trò chuyện cho người dùng
 - `Microsoft Learn Catalog API` để giúp người dùng tìm khóa học dựa trên yêu cầu của họ
 - `Function Calling` để lấy truy vấn của người dùng và gửi nó đến một hàm để thực hiện yêu cầu API.

Để bắt đầu, hãy xem tại sao chúng ta muốn sử dụng gọi hàm ngay từ đầu:

print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # nhận phản hồi mới từ GPT nơi nó có thể xem phản hồi của hàm


print(second_response.choices[0].message)


### Tại sao gọi hàm

Nếu bạn đã hoàn thành bất kỳ bài học nào khác trong khóa học này, có lẽ bạn đã hiểu được sức mạnh của việc sử dụng Mô hình Ngôn ngữ Lớn (LLMs). Hy vọng bạn cũng có thể thấy một số hạn chế của chúng.

Gọi hàm là một tính năng của Dịch vụ OpenAI được thiết kế để giải quyết các thách thức sau:

Định dạng phản hồi không nhất quán:
- Trước khi có gọi hàm, các phản hồi từ mô hình ngôn ngữ lớn thường không có cấu trúc và không nhất quán. Các nhà phát triển phải viết mã xác thực phức tạp để xử lý từng biến thể trong đầu ra.

Tích hợp hạn chế với dữ liệu bên ngoài:
- Trước tính năng này, việc kết hợp dữ liệu từ các phần khác của ứng dụng vào ngữ cảnh trò chuyện rất khó khăn.

Bằng cách chuẩn hóa định dạng phản hồi và cho phép tích hợp liền mạch với dữ liệu bên ngoài, gọi hàm đơn giản hóa việc phát triển và giảm nhu cầu về logic xác thực bổ sung.

Người dùng không thể nhận được câu trả lời như "Thời tiết hiện tại ở Stockholm là gì?". Điều này là do các mô hình bị giới hạn ở thời điểm dữ liệu được huấn luyện.

Hãy xem ví dụ dưới đây minh họa cho vấn đề này:

Giả sử chúng ta muốn tạo một cơ sở dữ liệu về dữ liệu sinh viên để có thể đề xuất khóa học phù hợp cho họ. Dưới đây là hai mô tả về sinh viên rất giống nhau về dữ liệu mà chúng chứa.


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finishing his studies."

Chúng tôi muốn gửi điều này đến một LLM để phân tích dữ liệu. Điều này sau đó có thể được sử dụng trong ứng dụng của chúng tôi để gửi đến một API hoặc lưu trữ trong cơ sở dữ liệu.

Hãy tạo hai lời nhắc giống hệt nhau mà chúng tôi hướng dẫn LLM về thông tin mà chúng tôi quan tâm:


Chúng tôi muốn gửi điều này đến một LLM để phân tích các phần quan trọng đối với sản phẩm của chúng tôi. Vì vậy, chúng tôi có thể tạo hai lời nhắc giống hệt nhau để hướng dẫn LLM:


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


Sau khi tạo hai lời nhắc này, chúng tôi sẽ gửi chúng đến LLM bằng cách sử dụng `openai.ChatCompletion`. Chúng tôi lưu lời nhắc trong biến `messages` và gán vai trò là `user`. Điều này nhằm mô phỏng một tin nhắn từ người dùng được viết cho chatbot.


In [ ]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI()

deployment="gpt-3.5-turbo"

: 

Bây giờ chúng ta có thể gửi cả hai yêu cầu đến LLM và kiểm tra phản hồi mà chúng ta nhận được.


In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

Mặc dù các lời nhắc giống nhau và các mô tả tương tự, chúng ta có thể nhận được các định dạng khác nhau của thuộc tính `Grades`.

Nếu bạn chạy ô trên nhiều lần, định dạng có thể là `3.7` hoặc `3.7 GPA`.

Điều này là do LLM lấy dữ liệu không có cấu trúc dưới dạng lời nhắc được viết và cũng trả về dữ liệu không có cấu trúc. Chúng ta cần có một định dạng có cấu trúc để biết được điều gì sẽ xảy ra khi lưu trữ hoặc sử dụng dữ liệu này.

Bằng cách sử dụng gọi hàm chức năng, chúng ta có thể đảm bảo nhận lại dữ liệu có cấu trúc. Khi sử dụng gọi hàm chức năng, LLM thực sự không gọi hoặc chạy bất kỳ hàm nào. Thay vào đó, chúng ta tạo ra một cấu trúc để LLM tuân theo trong các phản hồi của nó. Sau đó, chúng ta sử dụng các phản hồi có cấu trúc đó để biết hàm nào cần chạy trong các ứng dụng của mình.


![Sơ đồ luồng gọi hàm](../../../../translated_images/Function-Flow.083875364af4f4bb69bd6f6ed94096a836453183a71cf22388f50310ad6404de.vi.png)


Chúng ta sau đó có thể lấy kết quả trả về từ hàm và gửi lại cho LLM. LLM sẽ phản hồi bằng ngôn ngữ tự nhiên để trả lời câu hỏi của người dùng.


### Các trường hợp sử dụng cho việc gọi hàm

**Gọi công cụ bên ngoài**  
Chatbot rất giỏi trong việc cung cấp câu trả lời cho các câu hỏi từ người dùng. Bằng cách sử dụng gọi hàm, chatbot có thể dùng các tin nhắn từ người dùng để hoàn thành một số tác vụ nhất định. Ví dụ, một sinh viên có thể yêu cầu chatbot "Gửi email cho giảng viên của tôi nói rằng tôi cần thêm sự trợ giúp về môn học này". Điều này có thể thực hiện một cuộc gọi hàm tới `send_email(to: string, body: string)`

**Tạo truy vấn API hoặc cơ sở dữ liệu**  
Người dùng có thể tìm thông tin bằng ngôn ngữ tự nhiên được chuyển đổi thành truy vấn hoặc yêu cầu API có định dạng. Một ví dụ có thể là một giáo viên yêu cầu "Ai là những sinh viên đã hoàn thành bài tập cuối cùng" có thể gọi một hàm tên là `get_completed(student_name: string, assignment: int, current_status: string)`

**Tạo dữ liệu có cấu trúc**  
Người dùng có thể lấy một đoạn văn bản hoặc CSV và sử dụng LLM để trích xuất thông tin quan trọng từ đó. Ví dụ, một sinh viên có thể chuyển đổi một bài viết Wikipedia về các thỏa thuận hòa bình để tạo thẻ học AI. Việc này có thể được thực hiện bằng cách sử dụng một hàm gọi là `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)`


## 2. Tạo Lời Gọi Hàm Đầu Tiên Của Bạn

Quá trình tạo một lời gọi hàm bao gồm 3 bước chính:
1. Gọi API Chat Completions với danh sách các hàm của bạn và một tin nhắn từ người dùng
2. Đọc phản hồi của mô hình để thực hiện một hành động, ví dụ như thực thi một hàm hoặc gọi API
3. Thực hiện một lần gọi khác tới API Chat Completions với phản hồi từ hàm của bạn để sử dụng thông tin đó tạo ra phản hồi cho người dùng.


![Luồng của một Cuộc gọi Hàm](../../../../translated_images/LLM-Flow.3285ed8caf4796d7343c02927f52c9d32df59e790f6e440568e2e951f6ffa5fd.vi.png)


### Các thành phần của một cuộc gọi hàm

#### Đầu vào của người dùng

Bước đầu tiên là tạo một tin nhắn người dùng. Điều này có thể được gán động bằng cách lấy giá trị từ một ô nhập văn bản hoặc bạn có thể gán một giá trị tại đây. Nếu đây là lần đầu tiên bạn làm việc với Chat Completions API, chúng ta cần định nghĩa `role` và `content` của tin nhắn.

`role` có thể là `system` (tạo quy tắc), `assistant` (mô hình) hoặc `user` (người dùng cuối). Đối với cuộc gọi hàm, chúng ta sẽ gán giá trị này là `user` và một câu hỏi ví dụ.


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### Tạo hàm.

Tiếp theo chúng ta sẽ định nghĩa một hàm và các tham số của hàm đó. Chúng ta sẽ chỉ sử dụng một hàm ở đây gọi là `search_courses` nhưng bạn có thể tạo nhiều hàm khác nhau.

**Quan trọng**: Các hàm được bao gồm trong tin nhắn hệ thống gửi đến LLM và sẽ được tính vào số lượng token khả dụng mà bạn có.


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**Định nghĩa**

Cấu trúc định nghĩa hàm có nhiều cấp độ, mỗi cấp độ có các thuộc tính riêng. Dưới đây là phân tích cấu trúc lồng nhau:

**Thuộc tính Hàm Cấp Đầu:**

`name` - Tên của hàm mà chúng ta muốn gọi.

`description` - Đây là mô tả về cách hàm hoạt động. Ở đây điều quan trọng là phải cụ thể và rõ ràng.

`parameters` - Danh sách các giá trị và định dạng mà bạn muốn mô hình tạo ra trong phản hồi của nó.

**Thuộc tính Đối tượng Tham số:**

`type` - Kiểu dữ liệu của đối tượng tham số (thường là "object")

`properties` - Danh sách các giá trị cụ thể mà mô hình sẽ sử dụng cho phản hồi của nó.

**Thuộc tính Tham số Cá nhân:**

`name` - Được định nghĩa ngầm định bởi khóa thuộc tính (ví dụ: "role", "product", "level")

`type` - Kiểu dữ liệu của tham số cụ thể này (ví dụ: "string", "number", "boolean")

`description` - Mô tả về tham số cụ thể.

**Thuộc tính Tùy chọn:**

`required` - Một mảng liệt kê các tham số bắt buộc để cuộc gọi hàm được hoàn thành.


### Gọi hàm  
Sau khi định nghĩa một hàm, bây giờ chúng ta cần đưa nó vào trong lời gọi API Chat Completion. Chúng ta làm điều này bằng cách thêm `functions` vào yêu cầu. Trong trường hợp này là `functions=functions`.  

Cũng có một tùy chọn để đặt `function_call` thành `auto`. Điều này có nghĩa là chúng ta sẽ để LLM quyết định hàm nào nên được gọi dựa trên tin nhắn của người dùng thay vì tự gán.


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

Bây giờ hãy cùng xem phản hồi và cách nó được định dạng:

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

Bạn có thể thấy tên của hàm được gọi và từ tin nhắn của người dùng, LLM đã có thể tìm dữ liệu phù hợp với các đối số của hàm.


## 3.Tích hợp các cuộc gọi hàm vào một ứng dụng. 


Sau khi chúng ta đã kiểm tra phản hồi được định dạng từ LLM, bây giờ chúng ta có thể tích hợp điều này vào một ứng dụng. 

### Quản lý luồng 

Để tích hợp điều này vào ứng dụng của chúng ta, hãy thực hiện các bước sau: 

Đầu tiên, hãy gọi dịch vụ OpenAI và lưu tin nhắn vào một biến có tên `response_message`. 


In [ ]:
response_message = response.choices[0].message

Bây giờ chúng ta sẽ định nghĩa hàm sẽ gọi API Microsoft Learn để lấy danh sách các khóa học:


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



Như một thực hành tốt nhất, chúng ta sẽ kiểm tra xem mô hình có muốn gọi một hàm hay không. Sau đó, chúng ta sẽ tạo một trong các hàm có sẵn và khớp nó với hàm đang được gọi.  
Tiếp theo, chúng ta sẽ lấy các đối số của hàm và ánh xạ chúng với các đối số từ LLM.

Cuối cùng, chúng ta sẽ thêm tin nhắn gọi hàm và các giá trị được trả về bởi tin nhắn `search_courses`. Điều này cung cấp cho LLM tất cả thông tin cần thiết để  
phản hồi người dùng bằng ngôn ngữ tự nhiên.


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



Bây giờ chúng ta sẽ gửi tin nhắn đã cập nhật đến LLM để chúng ta có thể nhận được phản hồi bằng ngôn ngữ tự nhiên thay vì phản hồi định dạng JSON của API.


In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## Thử thách mã 

Tuyệt vời! Để tiếp tục học về Gọi Hàm OpenAI, bạn có thể xây dựng: https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst 
 - Thêm các tham số của hàm có thể giúp người học tìm nhiều khóa học hơn. Bạn có thể tìm các tham số API có sẵn tại đây: 
 - Tạo một cuộc gọi hàm khác lấy thêm thông tin từ người học như ngôn ngữ mẹ đẻ của họ 
 - Tạo xử lý lỗi khi cuộc gọi hàm và/hoặc cuộc gọi API không trả về khóa học phù hợp nào


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Tuyên bố từ chối trách nhiệm**:  
Tài liệu này đã được dịch bằng dịch vụ dịch thuật AI [Co-op Translator](https://github.com/Azure/co-op-translator). Mặc dù chúng tôi cố gắng đảm bảo độ chính xác, xin lưu ý rằng bản dịch tự động có thể chứa lỗi hoặc không chính xác. Tài liệu gốc bằng ngôn ngữ gốc của nó nên được coi là nguồn tham khảo chính thức. Đối với các thông tin quan trọng, nên sử dụng dịch vụ dịch thuật chuyên nghiệp do con người thực hiện. Chúng tôi không chịu trách nhiệm về bất kỳ sự hiểu lầm hoặc giải thích sai nào phát sinh từ việc sử dụng bản dịch này.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
